In [1]:
from utz import *

In [2]:
cs = read_parquet('data/2020/NewJersey2020Accidents.pqt')
vs = read_parquet('data/2020/NewJersey2020Vehicles.pqt')
cs

,County Code,Municipality Code,Department Case Number,County Name,Municipality Name,Police Dept Code,Police Department,Police Station,Total Killed,Total Injured,Pedestrians Killed,Pedestrians Injured,Severity,Intersection,Alcohol Involved,HazMat Involved,Crash Type Code,Total Vehicles Involved,Crash Location,Location Direction,Route,Route Suffix,SRI (Std Rte Identifier),MilePost,Road System,Road Character,Road Horizontal Alignment,Road Grade,Road Surface Type,Surface Condition,Light Condition,Environmental Condition,Road Divided By,Temporary Traffic Control Zone,Distance To Cross Street,Unit Of Measurement,Directn From Cross Street,Cross Street Name,Is Ramp,Ramp To/From Route Name,Ramp To/From Route Direction,Posted Speed,Posted Speed Cross Street,First Harmful Event,Latitude,Longitude,Cell Phone In Use Flag,Other Property Damage,Reporting Badge No.,Date
0,01,01,2020-020639,ATLANTIC,ABSECON CITY,01,ABSECON CITY PD,,0,0,0,0,P,B,True,False,04,2,ATLANTIC COUNTY 663,,663,,01000663__,1.34,05,,01,04,02,01,07,01,05,01,10,FE,E,ROUTE 585 / N MAIN ST / S SHORE RD,,,,25,35,26,NaN,NaN,N,,861,2020-08-03 02:31:00
1,01,01,2020-021265,ATLANTIC,ABSECON CITY,01,ABSECON CITY PD,,0,0,0,0,P,I,False,False,03,2,US 30,E,30,,00000030__,52.30,02,,02,06,02,01,01,01,01,01,,AT,,CR 631 / ILLINOIS AVE / TURNER AVE,,,,45,25,26,NaN,NaN,N,,861,2020-08-10 17:47:00
2,01,01,2020005031,ATLANTIC,ABSECON CITY,01,PLEASANTVILLE PD,,0,0,0,0,P,I,False,False,03,2,US 9,S,9,,00000009__,41.87,02,,01,04,02,01,01,01,05,01,,AT,,CR 663 / CALIFORNIA AVE / W CALIFOR,,,,30,25,26,39.413761,74.512961,N,,1745,2020-02-29 16:54:00
3,01,01,2020025300,ATLANTIC,ABSECON CITY,01,EGG HARBOR TWP PD,EHTPD HQ,0,0,0,0,P,B,False,False,11,1,ATLANTIC COUNTY 685,N,685,,01000685__,0.23,05,,01,04,02,01,06,01,05,01,1200,FE,N,CR 651 / MILL RD,,,,45,45,52,39.435797,74.536607,N,,5191,2020-04-29 23:30:00
4,01,01,I-2020-000186,ATLANTIC,ABSECON CITY,01,ABSECON CITY PD,,0,0,0,0,P,B,False,False,11,1,US 30,,30,,00000030__,54.26,02,,01,04,02,01,01,01,01,01,882,FE,E,CR 646 / DELILAH RD,,,,50,25,19,NaN,NaN,N,,847,2020-01-02 08:35:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195469,21,23,B150-2020-00350A,WARREN,WHITE TWP,02,NEW JERSEY STATE POLICE,WASHINGTON - SQ,0,0,0,0,P,B,False,False,04,3,ROUTE 519,S,519,,00000519__,42.80,05,,02,04,02,03,01,03,05,01,,,,,,,,50,,26,40.812200,75.049400,N,,8246,2020-12-09 11:26:00
195470,21,23,B150-2020-00354A,WARREN,WHITE TWP,02,NEW JERSEY STATE POLICE,WASHINGTON - SQ,0,0,0,0,P,B,False,False,11,1,PARKING LOT OF 187 CR 519,,,,,NaN,09,,01,04,02,02,06,04,05,01,,,,,,,,15,,56,40.805720,75.076940,N,Mobile advertisement sign and concrete column.,8354,2020-12-12 18:45:00
195471,21,23,B150-2020-00361A,WARREN,WHITE TWP,02,NEW JERSEY STATE POLICE,WASHINGTON - SQ,1,0,1,0,F,I,False,False,13,1,ROUTE 519,S,519,,00000519__,38.12,05,,01,04,02,01,05,01,05,01,,AT,,WINDTRYST WAY,,,,50,25,22,40.863590,75.014050,N,,6657,2020-12-15 06:12:00
195472,21,23,B150-2020-00369A,WARREN,WHITE TWP,02,NEW JERSEY STATE POLICE,WASHINGTON - SQ,0,0,0,0,P,B,False,False,12,1,ROUTE 519,S,519,,00000519__,37.00,05,,01,04,02,01,05,01,05,01,,,,,,,,50,,24,40.746800,75.137400,N,,8246,2020-12-19 21:37:00


<details><summary><h2>Crash Type Codes, <a href="https://www.state.nj.us/transportation/refdata/accident/pdf/NJTR-1CrashReportManual.pdf">2017 manual</a>, pg 67:</h2></summary>

> ### CRASH TYPE
> 
> With other MV as first event
> - 01 - Same Direction (Read-End)
> - 02 - Same Direction (Side Swipe)
> - 03 - Right Angle
> - 04 - Opposite Direction (Head on, Angular)
> - 05 - Opposite Direction (Side Swipe)
> - 06 - Struck Parked Vehicle
> - 07 - Left Turn/U-turn
> - 08 - Backing
> - 09 - Encroachment
> 
> With below as first event
> - 10 - Overturn
> - 11 - Fixed Object
> - 12 - Animal
> - 13 - Pedestrian
> - 14 - Pedalcyclist
> - 15 - Non-fixed Object
> - 16 - Railcar Vehicle
</details>

In [38]:
cs['Crash Type Code'].value_counts().sort_index()

         81
00       38
01    51335
02    28672
03    27526
04     3064
05     1886
06    23068
07     3908
08     8669
09      746
10     1231
11    26545
12     9895
13     3041
14     1791
15     2464
16       13
99     1501
Name: Crash Type Code, dtype: int64

In [3]:
vs

,Year,County Code,Municipality Code,Department Case Number,Vehicle Number,Insurance Company Code,Owner State,Make of Vehicle,Model of Vehicle,Color of Vehicle,Year of Vehicle,License Plate State,Vehicle Weight Rating,Towed,Removed By,Driven/Left at Scene/Towed,Initial Impact Location,Principal Damage Location,Extent of Damage,Traffic Controls Present,Vehicle Type,Vehicle Use,Special Function Vehicles,Cargo Body Type,Contributing Circumstances 1,Contributing Circumstances 2,Direction of Travel,Pre-Crash Action,First Sequence of Events,Second Sequence of Events,Third Sequence of Events,Fourth Sequence of Events,Most Harmful Event,Oversize/Overweight Permit,HazMat Status,HazMat Class,HazMat Placard,USDOT Number,MC/MX Number,USDOT / Other Flag,USDOT / OTHER Number,Carrier Name,Hit & Run Driver Flag
0,2020,01,01,2020-020639,1,946,NJ,KIA,FORTE FE,BK,2020,NJ,,,3,5,11,11,04,08,01,01,,,05,02,02,01,26,,,,26,,,,,,,,,,
1,2020,01,01,2020-020639,2,100,NJ,LEXUS,GS 350,WT,2014,NJ,,,,1,12,01,03,08,01,01,,,25,,04,07,26,,,,26,,,,,,,,,,
2,2020,01,01,2020-021265,1,413,NJ,CHRYSLER,TOWN &,BK,2006,NJ,,,,1,01,01,03,03,01,01,,,04,,01,03,26,,,,26,,,,,,,,,,
3,2020,01,01,2020-021265,2,012,NJ,DODGE,CHALLENGER,RD,2019,NJ,,,,1,10,10,03,03,01,01,,,25,,02,02,26,,,,26,,,,,,,,,,
4,2020,01,01,2020005031,1,962,NJ,FORD,FUS,BK,2016,NJ,,,1,1,11,11,02,03,01,01,,,19,,02,02,26,,,,26,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362220,2020,21,23,B150-2020-00354A,1,134,NJ,CHEVROLET,BLAZER,SL,1996,NJ,,,1,1,11,17,01,11,01,01,,,99,,01,09,06,56,39,54,54,,,,,,,,,,Y
362221,2020,21,23,B150-2020-00361A,1,655,NJ,HONDA,ELEMENT,BK,2009,NJ,,,3,5,12,12,04,04,01,01,,,25,,03,01,22,,,,22,,,0,,,,,,,
362222,2020,21,23,B150-2020-00369A,1,99,PA,DODGE,1500,WT,2013,PA,,,1,1,11,08,02,04,05,01,,,57,,03,01,24,,,,24,,,,,,,,,,
362223,2020,21,23,B150-2020-00383A,1,191,IL,VOLVO,TRACTOR,WT,2019,IL,3,,2,1,03,03,03,03,23,02,,05,04,08,02,03,26,,,,26,,,0,,2431698,,,,SARGENT LOGISTICS,


In [10]:
cmd_hist = vs[['County Code', 'Municipality Code', 'Department Case Number']].value_counts()
cmd_hist

County Code  Municipality Code  Department Case Number
09           08                 20L542                    11
13           52                 2020-02341                11
07           14                 C20039126                 10
                                C20053860                 10
02           31                 2020-017310                9
                                                          ..
08           02                 A310-2020-00353A           1
02           66                 I-2020-007337              1
12           07                 I-2020-003967              1
05           04                 A050-2020-00070A           1
03           07                 2020-4001                  1
Length: 195464, dtype: int64

Expectation is that "vehicle" records are numbered \[1,N\] for each {County, Muni, Dept Case Number} (which is the primary key for a "crash" record):

In [40]:
%%time
def check_vehicle_numbers(df):
    if df['Vehicle Number'].astype(int).tolist() != list(range(1, len(df) + 1)):
        return ','.join(df['Vehicle Number'].tolist())
    else:
        return nan

bad_vn_groups = vs.groupby(['County Code', 'Municipality Code', 'Department Case Number']).apply(check_vehicle_numbers).dropna()
bad_vn_groups.value_counts()

CPU times: user 8.29 s, sys: 438 ms, total: 8.72 s
Wall time: 8.67 s


2          292
3           47
1,3         11
3,4          8
2,3          3
1,3,4        2
1,2,5,6      2
1,2,3,5      1
1,2,5        1
1,4          1
dtype: int64

In [19]:
cc = cs[['County Code', 'Municipality Code', 'Department Case Number']].value_counts()
cc

County Code  Municipality Code  Department Case Number
01           01                 2020-020639               1
13           37                 20OT15368                 1
                                20OT15123                 1
                                20OT15135                 1
                                20OT15185                 1
                                                         ..
07           14                 C20035011                 1
                                C20035013                 1
                                C20035014                 1
                                C20035016                 1
21           23                 B150-2020-00383A          1
Length: 195474, dtype: int64

In [50]:
pi = cs['Pedestrians Injured'].sum()
pk = cs['Pedestrians Killed'].sum()
ti = cs['Total Injured'].sum()
tk = cs['Total Killed'].sum()
tk, ti, pk, pi, pi/pk, ti/tk

(611, 58647, 173, 2677, 15.473988439306359, 95.98527004909984)

In [45]:
cs.Severity.value_counts()

P    151909
I     42989
F       576
Name: Severity, dtype: int64

In [47]:
cs['Total Injured'].sum()

58647

In [12]:
vs.set_index(['County Code', 'Municipality Code', 'Department Case Number']).loc[('09', '08', '20L542')]

Year Vehicle Number  \
County Code Municipality Code Department Case Number                        
09          08                20L542                  2020              1   
                              20L542                  2020              2   
                              20L542                  2020              3   
                              20L542                  2020              4   
                              20L542                  2020              5   
                              20L542                  2020              6   
                              20L542                  2020              7   
                              20L542                  2020              8   
                              20L542                  2020              9   
                              20L542                  2020             10   
                              20L542                  2020             11   

                                                     Insurance Company Code  \
County Code Municipality Code Department Case Number                          
09          08                20L542                                    100   
                              20L542                                    217   
                              20L542                                          
                              20L542                                    963   
                              20L542                                    134   
                              20L542                                   2786   
                              20L542                                    426   
                              20L542                                    100   
                              20L542                                    228   
                              20L542                                    100   
                              20L542                                    134   

                                                     Owner State  \
County Code Municipality Code Department Case Number               
09          08                20L542                          NY   
                              20L542                          NJ   
                              20L542                          OK   
                              20L542                          NJ   
                              20L542                          NJ   
                              20L542                          MD   
                              20L542                          NJ   
                              20L542                          NJ   
                              20L542                          NY   
                              20L542                          NJ   
                              20L542                          NJ   

                                                     Make of Vehicle  \
County Code Municipality Code Department Case Number                   
09          08                20L542                          TOYOTA   
                              20L542                   MERCEDES BENZ   
                              20L542                          NISSAN   
                              20L542                           VOLVO   
                              20L542                             BMW   
                              20L542                            FORD   
                              20L542                           HONDA   
                              20L542                           HONDA   
                              20L542                          NISSAN   
                              20L542                          NISSAN   
                              20L542                          SUBARU   

                                                     Model of Vehicle  \
County Code Municipality Code Department Case Number                    
09          08                20L542                    

<details><summary><h2>Pre-Crash Action, <a href="https://www.state.nj.us/transportation/refdata/accident/pdf/NJTR-1CrashReportManual.pdf">2017 manual</a>, pg 86:</h2></summary>

> #### Vehicle/Pedalcyclist Action (01-29)
> - 01 - Going Straight Ahead
> - 02 - Making Right Turn (not turn on red)
> - 03 - Making Left Turn
> - 04 - Making U Turn
> - 05 - Starting From Parking
> - 06 - Starting In Traffic
> - 07 - Slowing or Stopping
> - 08 - Stopped In Traffic
> - 09 - Parking
> - 10 - Parked
> - 11 - Changing Lanes
> - 12 - Merging/Entering Traffic Lane
> - 13 - Backing
> - 14 - Driverless/Moving
> - 15 - Passing
> - 16 - Negotiating Curve
> - 17 - Driving on Shoulder
> - 18 - Right Turn on Red Signal
> - 19 - Deliberate Action*
> - 29 - Other Veh/Cyclist Action*
>
> #### Pedestrian Action (31-49)
> - 31 - Pedestrian Off Road
> - 32 - Walking To/From School
> - 33 - Walking/Jogging On Road W/Traffic
> - 34 - Walking/Jogging On Road Against Traffic
> - 35 - Playing In Road
> - 36 - Standing/Lying/Kneeling In Road
> - 37 - Getting On or Off Vehicle
> - 38 - Pushing or Working On Vehicle
> - 39 - Other Working In Roadway
> - 40 - Approaching or Leaving School Bus
> - 41 - Coming From Behind Parked Vehicle
> - 42 - Crossing / Jaywalking
> - 43 - Crossing at “Marked” Crosswalk at intersection
> - 44 - Crossing at “Unmarked” Crosswalk at intersection
> - 45 - Crossing at “Marked” Crosswalk at Mid-Block
> - 46 - Deliberate Action*
> - 49 - Other Pedestrian Action*
</details>

In [63]:
vs['Pre-Crash Action'].value_counts().sort_index()

        1185
00      5001
01    167456
02     11104
03     26267
04      1569
05      2048
06      7715
07     26294
08     33533
09      2367
10     33676
11     12849
12      7684
13     11608
14       303
15      2767
16      7180
17       214
18       525
19       264
29       423
99       193
Name: Pre-Crash Action, dtype: int64

<details><summary><h2>Most Harmful Event / Sequence of Events, <a href="https://www.state.nj.us/transportation/refdata/accident/pdf/NJTR-1CrashReportManual.pdf">2017 manual</a>, pg 90:</h2></summary>

> ### SEQUENCE OF EVENTS
>
> #### Non-Collision 01 – 19
> - 01 - Overturn/Rollover
> - 02 - Fire/Explosion
> - 03 - Immersion
> - 04 - Jackknife
> - 05 - Ran Off Road- Right**
> - 06 - Ran Off Road- Left**
> - 07 - Cross Median**
> - 08 - Crossed Centerline**
> - 09 - Cargo/Equip Loss or Shift
> - 10 - Separation of Units**
> - 11 - Fell/Jumped From Vehicle
> - 12 - Thrown/Falling Object
> - 13 - Equipment Failure (blown tire, brake failure etc.)**
> - 14 - Downhill Runaway**
> - 15 - Reentered Roadway**
> - 19 - Other Non-Collision*
>
> #### Collision w/Person, MV or Non-Fixed Object 21– 39 
> - 21 - Pedalcyclist
> - 22 - Pedestrian
> - 23 - Train/Trolley/Other Railcar
> - 24 - Deer
> - 25 - Other Animal
> - 26 - MV in Transport
> - 27 - MV in Transport, Other Roadway
> - 28 - Parked MV
> - 29 - Work Zone/Maintenance Equipment
> - 30 - Struck By Object Set in Motion By MV
> - 39 - Other Non-Fixed Object*
>
> #### Collision w/Fixed Object 41 - 69
> - 41 - Impact Attenuator/Crash Cushion
> - 42 - Bridge Overhead Structure
> - 43 - Bridge Pier or Support
> - 44 - Bridge Parapet End
> - 45 - Bridge Rail
> - 46 – Guide Rail Face
> - 47 – Guide Rail End
> - 48 - Concrete Traffic Barrier
> - 49 - Other Traffic Barrier
> - 50 - Traffic Sign Support
> - 51 - Traffic Signal Standard
> - 52 - Utility Pole
> - 53 - Light Standard
> - 54 - Other Post, Pole, Support
> - 55 - Culvert
> - 56 - Curb
> - 57 - Ditch
> - 58 - Embankment
> - 59 - Fence
> - 60 - Tree
> - 61 - Mailbox
> - 62 - Fire Hydrant
> - 69 - Other Fixed Object*
</details>

In [65]:
sxs(
    vs['Most Harmful Event'].value_counts().sort_index(),
    *[
    vs[f'{k} Sequence of Events'].value_counts().sort_index()
    for k in [ 'First', 'Second', 'Third', 'Fourth' ]
    ]
).fillna(0).astype(int).sort_index()

,Most Harmful Event,First Sequence of Events,Second Sequence of Events,Third Sequence of Events,Fourth Sequence of Events
,30432,634,309746,337887,347691
00,4810,204,5147,5244,5282
01,1818,454,1096,865,574
02,67,30,71,57,62
03,22,23,59,39,39
04,50,52,53,44,26
05,0,7677,3177,862,190
06,0,3684,2614,733,178
07,0,175,225,82,27
08,0,1975,695,127,53


<details><summary><h2>Contributing Circumstances, <a href="https://www.state.nj.us/transportation/refdata/accident/pdf/NJTR-1CrashReportManual.pdf">2017 manual</a>, pg 80:</h2></summary>

> ### Apparent Contributing Circumstances
> 
> #### Human/ Driver Actions 01 – 29
> - 01 - Unsafe Speed
> - 02 - Driver Inattention*
> - 03 - Failed to Obey Traffic Signal
> - 04 - Failed to Yield ROW to Vehicle/Pedestrian
> - 05 - Improper Lane Change
> - 06 - Improper Passing
> - 07 - Improper Use/Failed to Use turn signal
> - 08 - Improper Turning
> - 09 - Following Too Closely
> - 10 - Backing Unsafely
> - 11 - Improper use/no lights
> - 12 - Wrong Way
> - 13 - Improper Parking
> - 14 - Failure to Keep Right
> - 15 - Failure to remove Snow/Ice
> - 16 - Failure to Obey Stop Sign
> - 17 - Distracted – Hand Held Electronic Device*
> - 18 - Distracted – Hands Free Electronic Device*
> - 19 - Distracted by passenger*
> - 20 - Other Distraction Inside Vehicle*
> - 21 - Other Distraction Outside Vehicle*
> - 25 - None
> - 29 - Other Driver/Pedalcyclist Action*
>
> #### Vehicle Factors 31 – 49
> - 31 - Defective Lights
> - 32 - Brakes*
> - 33 - Steering*
> - 34 - Tire *
> - 35 - Wheels*
> - 36 - Windows/Windshield*
> - 37 - Mirrors
> - 38 - Wipers
> - 39 - Vehicle Coupling/Hitch/Safety Chains*
> - 49 - Other Vehicle Factor* 
>
> #### Road/Environmental Factors 51 – 69
> - 51 - Road Surface Condition*
> - 52 - Obstruction/Debris on Road*
> - 53 - Ruts, Holes, Bumps*
> - 54 - Traffic Control Device Defective/Missing*
> - 55 - Improper Work Zone*
> - 56 - Physical Obstruction(s) (viewing, etc)*
> - 57 - Animal(s) in Roadway*
> - 58 - Improper/Inadequate Lane Markings*
> - 59 - Sun Glare*
> - 60 - Traffic Congestion – Prior Incident*
> - 61 - Traffic Congestion – Regular*
> - 69 - Other Roadway Factors*
>
> #### Pedestrian Factors 71 - 89
> - 71 - Failed to obey Traffic control Device
> - 72 - Crossing where prohibited
> - 73 - Dark clothing/Low visibility to driver
> - 74 - Inattentive*
> - 75 - Failure to yield Right of Way
> - 76 - Walking on wrong side of road
> - 77 - Walking in road when sidewalk is present
> - 78 - Running/Darting Across Traffic
> - 85 - None
> - 89 - Other Pedestrian Factors*
</details>

In [52]:
vs['Contributing Circumstances 1'].value_counts().sort_index()

        7472
00     17993
01     10684
02     78596
03      3948
04     13222
05      9811
06      3731
07       201
08      4704
09     18848
10      6119
11        69
12       313
13      1379
14      1387
15        34
16      2683
17       543
18       186
19       156
20      1125
21       883
25    152929
29      4201
31        30
32       806
33       352
34       634
35       262
36        38
37        10
38         5
39        69
40       290
49       998
51      2509
52      1535
53       146
54        73
55        17
56       238
57      9171
58         8
59       520
60       127
61       158
69       326
99      2686
Name: Contributing Circumstances 1, dtype: int64

In [55]:
vs['Contributing Circumstances 2'].value_counts().sort_index()

      286433
00     13380
01      2489
02     18858
03      1335
04      4394
05      2223
06      1062
07        87
08      2228
09      5100
10      2063
11        29
12       237
13       279
14       678
15         8
16       898
17       307
18       120
19       127
20       776
21       457
25     11221
29      1128
31        13
32       463
33       147
34       220
35        71
36        43
37        14
38         1
39         9
40        14
49       289
51      2237
52       150
53        54
54        32
55        10
56       191
57       405
58        14
59       533
60       104
61       262
69       163
99       869
Name: Contributing Circumstances 2, dtype: int64

<details><summary><h2>Extent of Damage, <a href="https://www.state.nj.us/transportation/refdata/accident/pdf/NJTR-1CrashReportManual.pdf">2017 manual</a>, pg 96:</h2></summary>

> 01 – None
>
> 02 – Minor - Damage that does not affect the operation of or disable the motor vehicle in transport.
>
> 03 – Moderate / Functional - Damage that is not disabling, but affects operation of the motor vehicle or
its parts.
>
> 04 – Disabling - Damage that precludes departure of the motor vehicle from the scene of the crash in its
usual daylight-operating manner after simple repairs. As a result, the motor vehicle had to be towed, or
carried from crash scene, or assisted by an emergency motor vehicle. im
</details>

In [32]:
vs['Extent of Damage'].value_counts().sort_index()

        6054
00     17797
01     13741
02    121096
03    109125
04     94412
Name: Extent of Damage, dtype: int64

In [41]:
vs['Hit & Run Driver Flag'].value_counts()

     334513
Y     27712
Name: Hit & Run Driver Flag, dtype: int64

In [27]:
vs['Towed'].value_counts()

    362225
Name: Towed, dtype: int64

In [28]:
vs['Removed By'].value_counts()

1    122963
2    120655
3     82281
      36326
Name: Removed By, dtype: int64

## "Driven/Left at Scene/Towed"
From [2017VehicleTable.pdf](data/fields/2017VehicleTable.pdf):
- 1 - Driven
- 2 - Left at Scene
- 3 - Towed Disabled
- 4 - Towed Impounded
- 5 - Towed Disabled & Impounded

In [30]:
vs['Driven/Left at Scene/Towed'].value_counts().sort_index()

      19100
1    213169
2     36653
3     81516
4      4122
5      7665
Name: Driven/Left at Scene/Towed, dtype: int64

In [66]:
vs['Vehicle Weight Rating'].value_counts().sort_index()

     337969
1      7888
2      4704
3     11664
Name: Vehicle Weight Rating, dtype: int64

In [67]:
make = vs['Make of Vehicle'].value_counts()
make

HONDA        46902
TOYOTA       40337
FORD         37692
NISSAN       29798
CHEVROLET    23610
             ...  
INDI             1
APPALACHI        1
VERMEER          1
MIDL             1
BBC              1
Name: Make of Vehicle, Length: 1317, dtype: int64

In [68]:
make.iloc[:100]

HONDA              46902
TOYOTA             40337
FORD               37692
NISSAN             29798
CHEVROLET          23610
                   16755
HYUNDAI            16406
JEEP               15255
DODGE              13558
BMW                 8878
KIA                 7838
SUBARU              7659
ACURA               7240
MERCEDES BENZ       7094
VOLKSWAGEN          6939
MAZDA               6646
LEXUS               6141
GMC                 5832
FREIGHTLINER        5472
CHRYSLER            4104
INFINITI            3893
AUDI                3888
VOLVO               3401
CADILLAC            2811
BUICK               2767
MITSUBISHI          2549
INTERNATIONAL       2377
LINCOLN             2115
MERCURY             1661
PETERBILT           1539
KENWORTH            1409
MACK                1396
PONTIAC             1090
LAND ROVER          1047
HINO                1034
SATURN               933
UNK                  922
RAM                  849
ISUZU                813
SUZUKI               648


In [71]:
(vs[['USDOT / OTHER Number', 'USDOT / Other Flag', 'USDOT Number']] != '').sum()

USDOT / OTHER Number        0
USDOT / Other Flag          0
USDOT Number            12725
dtype: int64

In [72]:
vs[vs['USDOT Number'] != '']

,Year,County Code,Municipality Code,Department Case Number,Vehicle Number,Insurance Company Code,Owner State,Make of Vehicle,Model of Vehicle,Color of Vehicle,Year of Vehicle,License Plate State,Vehicle Weight Rating,Towed,Removed By,Driven/Left at Scene/Towed,Initial Impact Location,Principal Damage Location,Extent of Damage,Traffic Controls Present,Vehicle Type,Vehicle Use,Special Function Vehicles,Cargo Body Type,Contributing Circumstances 1,Contributing Circumstances 2,Direction of Travel,Pre-Crash Action,First Sequence of Events,Second Sequence of Events,Third Sequence of Events,Fourth Sequence of Events,Most Harmful Event,Oversize/Overweight Permit,HazMat Status,HazMat Class,HazMat Placard,USDOT Number,MC/MX Number,USDOT / Other Flag,USDOT / OTHER Number,Carrier Name,Hit & Run Driver Flag
51,2020,01,01,I-2020-003369,2,100,NJ,DODGE,CAR,BK,2014,NJ,,,3,1,06,06,03,04,01,02,06,,25,,03,07,26,,,,26,,,,,2511146,871047,,,JOHNNY LIMO,
65,2020,01,01,I-2020-003806,1,,OK,HINO,268,WT,2017,PA,2,,2,1,06,06,02,04,20,02,,03,10,02,02,13,26,,,,26,,,,,2962496,,,,SHECKLEFORD & SIMMONS,
105,2020,01,01,I-2020-007019,2,487,NJ,MACK,RD6,GN,2003,NJ,3,,3,1,11,11,02,04,21,02,,,25,,04,01,26,,,,26,,,,,0446050,,,,WASTE MANAGEMENT OF NJ,
130,2020,01,01,I-2020-011671,2,,IN,FREIGHTLINER,CASC125,WT,2017,IN,3,,1,1,99,99,,04,23,02,,03,02,,01,07,26,,,,26,,,,,264184,,,,SCHNEIDER NATIONAL CARR,
156,2020,01,01,I-2020-014860,1,650,NJ,FORD,F45,WT,2004,NJ,2,,3,1,12,12,03,04,22,02,,,01,02,03,01,26,,,,26,,,,,3287871,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362073,2020,21,23,B150-2020-00012A,1,038,NJ,PETERBILT,99,BL,2016,NJ,3,,2,3,14,14,04,04,21,02,01,06,09,,03,01,05,58,,,58,01,,0,,2426280,,,,Jhonatan Trucking LLC,
362123,2020,21,23,B150-2020-00131A,1,656,NJ,FORD,F550,BL,2015,NJ,1,,1,1,01,01,02,03,21,02,16,05,05,,03,01,26,,,,26,01,,,,704700,,,,,
362128,2020,21,23,B150-2020-00142A,1,99,FL,INTERNATIONAL,TRUCK,WT,2020,IN,3,,1,1,08,08,02,04,20,02,20,03,25,,03,03,26,,,,26,,,0,,298201,,,,A PANZA AND SONS,
362171,2020,21,23,B150-2020-00249A,2,135,NJ,FREIGHTLINER,TRUCK,WT,2006,NJ,,,1,1,12,12,02,,21,02,,13,09,,03,01,26,,,,26,02,N,0,,2844858,,,,,


In [25]:
vs['License Plate State'].value_counts()

NJ    297114
       18571
PA     13898
NY     13449
FL      2301
IN      2192
MD      1797
TX      1211
CT      1097
VA      1078
DE      1051
IL       886
NC       883
MA       756
GA       604
OH       558
TN       496
AZ       490
CA       421
OK       387
SC       353
MI       239
MO       172
ME       165
WI       140
MN       139
NH       136
RI       130
IA       111
AL       100
WA        95
NE        86
ON        83
CO        81
MS        81
VT        80
KY        79
KS        67
WV        62
ZZ        62
LA        61
DC        60
AR        50
QU        46
ID        41
OR        39
NB        38
ND        35
NV        35
UT        31
NM        21
SD        19
MT        18
AK        13
WY         8
HI         8
MX         1
Name: License Plate State, dtype: int64

In [24]:
vs['Owner State'].value_counts()

NJ    286297
       25201
NY     13499
PA     13084
MD      2442
FL      2107
OK      2073
DE      1814
GA      1809
TX      1674
CA      1507
IL      1326
IN      1022
CT       975
VA       906
OH       852
NC       746
MA       604
AZ       498
TN       413
MO       358
WI       295
SC       276
MI       214
NH       200
WA       169
MN       160
AL       156
RI       136
AR       124
ME       109
IA       105
NE        93
KS        91
DC        90
CO        84
ON        80
MS        73
KY        67
VT        58
LA        50
QU        49
ZZ        44
UT        40
WV        40
NV        37
OR        32
MT        29
NB        26
ND        26
ID        17
SD        15
NM        13
AK        10
WY         5
HI         4
PR         1
Name: Owner State, dtype: int64